In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets
from efficientnet_pytorch import EfficientNet

# Define paths and hyperparameters
BATCH_SIZE = 64
NUM_EPOCHS = 100
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# define the EfficientNet models to use
# b0 max: 64
# b3 max: 32?
efficientnets = {
    "b0": EfficientNet.from_pretrained("efficientnet-b0", num_classes=5),
    # "b1": EfficientNet.from_pretrained("efficientnet-b1", num_classes=5),
    # "b2": EfficientNet.from_pretrained("efficientnet-b2", num_classes=5),
    # "b3": EfficientNet.from_pretrained("efficientnet-b3", num_classes=5),
    # "b4": EfficientNet.from_pretrained("efficientnet-b4", num_classes=5),
    # "b5": EfficientNet.from_pretrained("efficientnet-b5", num_classes=5),
    # "b6": EfficientNet.from_pretrained("efficientnet-b6", num_classes=5),
    # "b7": EfficientNet.from_pretrained("efficientnet-b7", num_classes=5),
}

efficientnet_sizes = {
    "b0": 224,
    "b1": 240,
    "b2": 260,
    "b3": 300,
    "b4": 380,
    "b5": 456,
    "b6": 528,
    "b7": 600,
}

Loaded pretrained weights for efficientnet-b0


In [2]:
from sklearn.model_selection import train_test_split
import time
from lion_pytorch import Lion

def train():
    # train the models and evaluate them on the validation set
    for model_name, model in efficientnets.items():
        # define the transform for data augmentation and resizing
        image_size = efficientnet_sizes[model_name]
        # Define data augmentations and transformations
        train_transforms = transforms.Compose(
            [
                transforms.Resize((image_size, image_size)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(20),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )
        val_transforms = transforms.Compose(
            [
                transforms.Resize((image_size, image_size)),
                # transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

        # Create train and validation datasets
        dataset = datasets.ImageFolder("data_sorted", transform=train_transforms)
        dataset_val = datasets.ImageFolder("data_sorted", transform=val_transforms)
        print(dataset.classes)

        train_idx, val_idx = train_test_split(
            list(range(len(dataset.targets))), test_size=0.2, stratify=dataset.targets
        )
        train_dataset = torch.utils.data.Subset(dataset, train_idx)
        val_dataset = torch.utils.data.Subset(dataset_val, val_idx)
        train_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, pin_memory=True
        )
        val_loader = torch.utils.data.DataLoader(
            val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, pin_memory=True
        )

        # Define model
        model.to(DEVICE)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = Lion(model.parameters(), lr=1e-4, weight_decay=1e-2)

        # Train and validate the model
        for epoch in range(NUM_EPOCHS):
            start = time.time()
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}")

            # Train the model
            model.train()
            train_loss = 0.0
            train_acc = 0.0
            for images, labels in train_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                train_loss += loss.item() * images.size(0)
                _, predictions = torch.max(outputs, 1)
                train_acc += torch.sum(predictions == labels.data)
            train_loss /= len(train_dataset)
            train_acc /= len(train_dataset)
            print(f"Train loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
            print(f"Train time: {time.time() - start}")

            start = time.time()
            # Validate the model
            model.eval()
            val_loss = 0.0
            val_acc = 0.0
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(DEVICE), labels.to(DEVICE)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item() * images.size(0)
                    _, predictions = torch.max(outputs, 1)
                    val_acc += torch.sum(predictions == labels.data)
            val_loss /= len(val_dataset)
            val_acc /= len(val_dataset)
            print(f"Val   loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

            # Save checkpoint
            checkpoint_path = f"checkpoint_{epoch+1}.pt"
            torch.save(
                {
                    "epoch": epoch + 1,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "train_loss": train_loss,
                    "train_acc": train_acc,
                    "val_loss": val_loss,
                    "val_acc": val_acc,
                },
                checkpoint_path,
            )

            # Save loss and accuracy values to file
            with open("loss_acc.txt", "a") as file:
                file.write(
                    f"{model_name}, {train_loss:.4f}, {train_acc:.4f}, {val_loss:.4f}, {val_acc:.4f}, {epoch}, {BATCH_SIZE}\n"
                )

            print(f"Val and misc time: {time.time() - start}")


if __name__ == "__main__":
    train()

['0', '1', '2', '3', '4']
Epoch 1/10
Train loss: 0.7704, Acc: 0.7460
Train time: 306.99389696121216
Val   loss: 0.9119, Acc: 0.7687
Val and misc time: 82.02202558517456
Epoch 2/10
Train loss: 0.6399, Acc: 0.7859
Train time: 297.63864612579346
Val   loss: 0.6494, Acc: 0.7800
Val and misc time: 83.00439262390137
Epoch 3/10


KeyboardInterrupt: 